# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [1]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [2]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        # set no pending order
        self.order = None


    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # cancel if there is an order pending, this strategy should have 1 working order per day
        if self.order:
            self.log('ORDER CANCELLED')
            self.cancel(self.order)
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
            self.buy(size=self.size) # market order buys at next open                      
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            # TODO: Make prediction and close a filter in the class constructor (more optimal)
            if data.prediction[0] >= data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')  
                self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
        # if prediction is less than current value sell at open (ASAP)
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.sell(size=self.size)
        

In [3]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [4]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 




## Import Data and Run Backtest

In [5]:
import os

In [6]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../../data/ticker_predictions/{pred_folder_name}'):
        # collect data
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        
        # make timestamp in column position 0, SPECIFIC TO FULL DATA IN FILE 
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']
        # merge prediction data with full data, on key = timestamp
        ticker_df = ticker_df.merge(pred_df.loc[:, ['timestamp', pred_col_name]], on='timestamp')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df = ticker_df.rename({pred_col_name:'prediction'}, axis=1)
        # remove all non-prediction rows in prediction field
        ticker_df = ticker_df[ticker_df.prediction > 0]
        # skip nonfeatures, keep timestamp, SPECIFIC TO FULL DATA IN FILE 
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]
        
        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict

In [8]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('arima_preds', 'prediction', '../../data/ticker_data')
cerebro_dict

--------Ticker Name-------: WAT
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-18, BUY EXECUTED, 228.60
Stop price: 349.9200134277344
ROI:        53.02%
Final Portfolio Value: 153016.8432006836

STRATEGY INFO:
OrderedDict([('sharperatio', 0.23066872516298156)])
AutoOrderedDict([('len', 43), ('drawdown', 17.598069851467034), ('moneydown', 32678.859466552793), ('max', AutoOrderedDict([('len', 227), ('drawdown', 33.451625540555426), ('moneydown', 37568.890533447324)]))])
OrderedDict([('rtot', 0.425377815624272), ('ravg', 0.0007848299181259631), ('rnorm', 0.2186907653606273), ('rnorm100', 21.869076536062728)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-17, Current Portfolio Value : 100000.0
2019-09-17, MARKET BUY CREATE 437 shares at next open, current close price: 228.69000244140625
2019-09-18, BUY EXECUTED, 228.60
2019-09-18, Current Portfolio Value : 99886.3757324219
2019-09-18, LIMIT SELL CREATE 437 shares at 230.13074705

2020-03-10, SELL EXECUTED, 195.76
2020-03-10, Current Portfolio Value : 83456.31820275064
2020-03-10, MARKET BUY CREATE 435 shares at next open, current close price: 196.72000122070312
2020-03-11, BUY EXECUTED, 191.78
2020-03-11, Current Portfolio Value : 80115.52138878578
2020-03-11, LIMIT SELL CREATE 435 shares at 185.5438058043336
2020-03-12, Current Portfolio Value : 72698.77006127115
2020-03-12, LIMIT SELL CREATE 435 shares at 168.3600885293876
2020-03-13, SELL EXECUTED, 177.42
2020-03-13, Current Portfolio Value : 77209.71793724771
2020-03-13, MARKET BUY CREATE 470 shares at next open, current close price: 184.47999572753903
2020-03-16, BUY EXECUTED, 164.20
2020-03-16, Current Portfolio Value : 81778.11851097818
2020-03-16, LIMIT SELL CREATE 470 shares at 175.2839614117352
2020-03-17, SELL EXECUTED, 177.19
2020-03-17, Current Portfolio Value : 83315.02051903482
2020-03-17, MARKET BUY CREATE 473 shares at next open, current close price: 183.2700042724609
2020-03-18, BUY EXECUTED, 

2020-09-10, BUY EXECUTED, 207.65
2020-09-10, Current Portfolio Value : 76926.40010458267
2020-09-10, LIMIT SELL CREATE 377 shares at 205.630098950015
2020-09-11, SELL EXECUTED, 205.63
2020-09-11, Current Portfolio Value : 77529.6378689434
2020-09-11, MARKET BUY CREATE 376 shares at next open, current close price: 203.92999267578125
2020-09-14, BUY EXECUTED, 206.01
2020-09-14, Current Portfolio Value : 77255.1594753887
2020-09-14, LIMIT SELL CREATE 376 shares at 206.889902043802
2020-09-15, SELL EXECUTED, 206.89
2020-09-15, Current Portfolio Value : 77860.48310284264
2020-09-15, MARKET BUY CREATE 376 shares at next open, current close price: 205.6100006103516
2020-09-16, BUY EXECUTED, 206.96
2020-09-16, Current Portfolio Value : 77721.35920147547
2020-09-16, LIMIT SELL CREATE 376 shares at 208.21017322553791
2020-09-17, Current Portfolio Value : 76672.32172588952
2020-09-17, LIMIT SELL CREATE 376 shares at 205.3982994867258
2020-09-18, Current Portfolio Value : 73540.24103741295
2020-09

2021-02-22, BUY EXECUTED, 280.60
2021-02-22, Current Portfolio Value : 98180.18509851769
2021-02-22, LIMIT SELL CREATE 350 shares at 282.0044004774824
2021-02-23, Current Portfolio Value : 97854.68766199426
2021-02-23, LIMIT SELL CREATE 350 shares at 281.06711435736366
2021-02-24, SELL EXECUTED, 281.07
2021-02-24, Current Portfolio Value : 98620.17597808717
2021-02-24, MARKET BUY CREATE 351 shares at next open, current close price: 279.8999938964844
2021-02-25, BUY EXECUTED, 280.51
2021-02-25, Current Portfolio Value : 97819.89640655396
2021-02-25, LIMIT SELL CREATE 351 shares at 280.4120228999767
2021-02-26, Current Portfolio Value : 96293.04426421998
2021-02-26, LIMIT SELL CREATE 351 shares at 276.0279019822156
2021-03-01, Current Portfolio Value : 97402.20554962037
2021-03-01, LIMIT SELL CREATE 351 shares at 279.2126878941385
2021-03-02, Current Portfolio Value : 96345.692121886
2021-03-02, LIMIT SELL CREATE 351 shares at 276.1790722020877
2021-03-03, Current Portfolio Value : 93449

2021-08-09, Current Portfolio Value : 114869.69038630712
2021-08-09, LIMIT SELL CREATE 291 shares at 396.7069501000496
2021-08-10, SELL EXECUTED, 396.71
2021-08-10, Current Portfolio Value : 115767.99428631998
2021-08-10, MARKET BUY CREATE 293 shares at next open, current close price: 394.8800048828125
2021-08-11, BUY EXECUTED, 395.08
2021-08-11, Current Portfolio Value : 116559.09786298014
2021-08-11, LIMIT SELL CREATE 293 shares at 400.8995784870021
2021-08-12, SELL EXECUTED, 400.90
2021-08-12, Current Portfolio Value : 117473.13471733777
2021-08-12, MARKET BUY CREATE 292 shares at next open, current close price: 401.3299865722656
2021-08-13, BUY EXECUTED, 402.03
2021-08-13, Current Portfolio Value : 118419.21186577527
2021-08-13, LIMIT SELL CREATE 292 shares at 408.4483087827622
2021-08-16, Current Portfolio Value : 119254.33649956434
2021-08-16, LIMIT SELL CREATE 292 shares at 411.3307542549408
2021-08-17, Current Portfolio Value : 118667.41364800184
2021-08-17, LIMIT SELL CREATE 2

...

2020-02-25, SELL EXECUTED, 196.84
2020-02-25, Current Portfolio Value : 117388.90833408065
2020-02-25, MARKET BUY CREATE 622 shares at next open, current close price: 186.72999572753903
2020-02-26, BUY EXECUTED, 188.59
2020-02-26, Current Portfolio Value : 118682.66947299665
2020-02-26, LIMIT SELL CREATE 622 shares at 193.09586783879152
2020-02-27, Current Portfolio Value : 112450.22681552597
2020-02-27, LIMIT SELL CREATE 622 shares at 182.94838035088128
2020-02-28, SELL EXECUTED, 182.95
2020-02-28, Current Portfolio Value : 113879.82319016082
2020-02-28, MARKET BUY CREATE 597 shares at next open, current close price: 190.8800048828125
2020-03-02, BUY EXECUTED, 190.67
2020-03-02, Current Portfolio Value : 113975.34537644012
2020-03-02, LIMIT SELL CREATE 597 shares at 193.2579072067524
2020-03-03, Current Portfolio Value : 107838.18610519989
2020-03-03, LIMIT SELL CREATE 597 shares at 182.8471173355992
2020-03-04, SELL EXECUTED, 183.90
2020-03-04, Current Portfolio Value : 109838.130639

2020-08-20, SELL EXECUTED, 244.49
2020-08-20, Current Portfolio Value : 154589.06965376734
2020-08-20, MARKET BUY CREATE 620 shares at next open, current close price: 249.1499938964844
2020-08-21, BUY EXECUTED, 249.33
2020-08-21, Current Portfolio Value : 153931.87116743918
2020-08-21, LIMIT SELL CREATE 620 shares at 251.4287113531811
2020-08-24, Current Portfolio Value : 156120.47041060327
2020-08-24, LIMIT SELL CREATE 620 shares at 255.0036218493476
2020-08-25, Current Portfolio Value : 156393.27192427518
2020-08-25, LIMIT SELL CREATE 620 shares at 255.44922238477764
2020-08-26, Current Portfolio Value : 153894.6726811111
2020-08-26, LIMIT SELL CREATE 620 shares at 251.36795045342691
2020-08-27, Current Portfolio Value : 150732.66889693143
2020-08-27, LIMIT SELL CREATE 620 shares at 246.20305763337828
2020-08-28, SELL EXECUTED, 246.20
2020-08-28, Current Portfolio Value : 152650.36425120797
2020-08-28, MARKET BUY CREATE 619 shares at next open, current close price: 247.38999938964844

2021-02-12, SELL EXECUTED, 304.03
2021-02-12, Current Portfolio Value : 149993.2469374637
2021-02-12, MARKET BUY CREATE 487 shares at next open, current close price: 305.1400146484375
2021-02-16, BUY EXECUTED, 307.95
2021-02-16, Current Portfolio Value : 150124.7315871219
2021-02-16, LIMIT SELL CREATE 487 shares at 312.1414443411816
2021-02-17, Current Portfolio Value : 147348.82564229768
2021-02-17, LIMIT SELL CREATE 487 shares at 306.36891161774207
2021-02-18, Current Portfolio Value : 146472.2315871219
2021-02-18, LIMIT SELL CREATE 487 shares at 304.5460228134187
2021-02-19, SELL EXECUTED, 304.55
2021-02-19, Current Portfolio Value : 148335.50410277437
2021-02-19, MARKET BUY CREATE 492 shares at next open, current close price: 305.3999938964844
2021-02-22, BUY EXECUTED, 301.46
2021-02-22, Current Portfolio Value : 146485.5943127353
2021-02-22, LIMIT SELL CREATE 492 shares at 301.48761087101184
2021-02-23, Current Portfolio Value : 142820.18830687593
2021-02-23, LIMIT SELL CREATE 492

2021-07-16, Current Portfolio Value : 141031.82241449808
2021-07-16, LIMIT SELL CREATE 480 shares at 297.0619859665885
2021-07-19, Current Portfolio Value : 138142.22768793555
2021-07-19, LIMIT SELL CREATE 480 shares at 290.96540541307644
2021-07-20, SELL EXECUTED, 290.97
2021-07-20, Current Portfolio Value : 139896.82345808728
2021-07-20, MARKET BUY CREATE 474 shares at next open, current close price: 296.1199951171875
2021-07-21, BUY EXECUTED, 294.82
2021-07-21, Current Portfolio Value : 143195.8594077943
2021-07-21, LIMIT SELL CREATE 474 shares at 305.61950658354124
2021-07-22, SELL EXECUTED, 305.62
2021-07-22, Current Portfolio Value : 145015.78610700613
2021-07-22, MARKET BUY CREATE 467 shares at next open, current close price: 308.510009765625
2021-07-23, BUY EXECUTED, 310.19
2021-07-23, Current Portfolio Value : 146164.60211652762
2021-07-23, LIMIT SELL CREATE 467 shares at 316.62779923950984
2021-07-26, Current Portfolio Value : 146617.59268659598
2021-07-26, LIMIT SELL CREATE 

{'WAT': <backtrader.cerebro.Cerebro at 0x7fbd985bba10>,
 'UNH': <backtrader.cerebro.Cerebro at 0x7fbd9870a790>,
 'HD': <backtrader.cerebro.Cerebro at 0x7fbda9220710>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x7fbda967d5d0>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7fbda9217050>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7fbdd8a43950>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  